# Exploratory Data Analysis (EDA)
## S&P 500 Historical Data Project

This notebook explores historical S&P 500 price data from Investing.com.

We will:

1. Inspect the raw dataset.
2. Create a “dirty” version to simulate real-world data issues.
3. Explore missing values, duplicates, outliers.
4. Visualize trends and distributions.
5. Prepare for the cleaning pipeline.

## 1. Dataset Description

The dataset contains historical S&P 500 price data downloaded from Investing.com.

- Original file: `sp500_raw.csv`
- Location: `data/raw/`
- Columns:
  - Date
  - Price
  - Open
  - High
  - Low
  - Vol. (we will drop this column)
  - Change %
- Rows: 5001

In [ ]:
import pandas as pd

df = pd.read_csv("../data/raw/sp500_raw.csv")

if 'Vol.' in df.columns:
    df = df.drop(columns=['Vol.'])

df.head()

## 2. Dataset Info

Check basic info: shape, data types, missing values, duplicates


In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

### 2.1. Convert Columns to Numeric
The numeric columns (Price, Open, High, Low) are read as strings because they contain commas.
We also convert 'Change %' to float by removing the percent sign.

In [ ]:
num_cols = ['Price', 'Open', 'High', 'Low']
for col in num_cols:
    df[col] = df[col].str.replace(',', '').astype(float)


df['Change %'] = df['Change %'].str.replace('%', '').astype(float)

df.dtypes
df.head()

## 3. Simulate a Dirty Dataset
To demonstrate a cleaning pipeline, we create a dirty dataset with:
- Duplicate rows
- Missing values
- Outliers
- Shuffled rows
- Column name inconsistencies


In [ ]:
import numpy as np

df_dirty = df.copy()

duplicates = df_dirty.sample(20, random_state=42)
df_dirty = pd.concat([df_dirty, duplicates], ignore_index=True)

nan_indices = np.random.choice(df_dirty.index, 30, replace=False)
df_dirty.loc[nan_indices, 'Price'] = np.nan

outlier_indices = np.random.choice(df_dirty.index, 5, replace=False)
df_dirty.loc[outlier_indices, 'High'] *= 5

df_dirty = df_dirty.sample(frac=1, random_state=1).reset_index(drop=True)

df_dirty.columns = [col + " " for col in df_dirty.columns]

df_dirty.to_csv("../data/raw/sp500_dirty.csv", index=False)

df_dirty.head()

## 4. Inspection of the Dirty Dataset

    

In [ ]:
df_dirty = pd.read_csv("../data/raw/sp500_dirty.csv")

In [ ]:
df_dirty.info()

In [ ]:
df_dirty.isna().sum()

In [ ]:
df_dirty.duplicated().sum()

## 5. Descriptive Statistics
Explore basic statistics to detect anomalies/outliers

In [ ]:
df_dirty.describe()

## 6. Visualize price trends and distributions

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,4))
plt.plot(pd.to_datetime(df_dirty['Date ']), df_dirty['Price '], marker='o', markersize=2)
plt.title("Price over Time (Dirty Dataset)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,4))
plt.hist(df_dirty['Price '].dropna(), bins=50)
plt.title("Price Distribution (Dirty Dataset)")
plt.xlabel("Price")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

## 7. Observations
- Missing values exist in Price (30 introduced)
- Duplicate rows exist (20 introduced)
- Outliers in High column
- Rows are shuffled
- Column names have extra spaces
- Dataset is ready for cleaning pipeline